# Python Notebook on Euclidean Clustering for LiDAR point cloud data

This python notebook presents a method for Euclidean clustering of LiDAR point cloud data. The method is simple and efficient, and is able to accurately cluster the point cloud data into different objects. The method can be used in a variety of applications, such as object detection, scene segmentation, and tracking.

## Installation of Necessary Libraries

Inorder to work conveniently and access the 3 Dimensional Data of LiDAR, we would need plotly and open3d.

In [ ]:
!pip install pandas
!pip install plotly
!pip install open3d

## Mounting Google Drive onto the python Notebook to access the data sheet.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importing Installed libraries into the notebook for access in following code block.

In [ ]:
import pandas as pd
import copy
import plotly.graph_objects as go
import open3d as o3d
import math

## Establishing a Node Class

Making a list of [X,Y,Z] Coordinates

In [ ]:
class Node:
    def __init__(self, data, point_id):
        self.point = {"X": data[0], "Y": data[1], "Z": data[2]}
        self.point_id = point_id
        self.left_node = None
        self.right_node = None

## Implementation of KdTree Class

A k-d tree (short for k-dimensional tree) is a space-partitioning data structure for organizing points in a k-dimensional space. K-dimensional is that which concerns exactly k orthogonal axes or a space of any number of dimensions.

In [ ]:
class KdTree_class:
    def __init__(self):
        self.root = None
        self.kdtree_display_dict = dict()

    def insert_points(self, pcd_dataframe, display_output=False):
        """
        :parameter pcd_dataframe: dataframe containing (x,y,z) coordinates data
        :return:
        """
        for row in pcd_dataframe.iterrows():
            x, y, z, point_id = row[1]["X"], row[1]["Y"], row[1]["Z"], row[0]
            point = (x, y, z)
            level = 0
            self.root = self.build_kdtree(self.root, level, point, point_id)


        if display_output:
            print("Kdtree Build Complete")
            self.display_kdtree(self.root)
            for pair in self.kdtree_display_dict.items():
                print(f"Depth = {pair[0]}, Points = {pair[1]} ")
        return self.root

    def build_kdtree(self, node, depth, point,point_id):
        """
        :parameter node: Node class object
        :parameter depth: level0 -x, level1-y, Level2-z
        :return: root node
        """
        if node is None:
            node = Node(point, point_id)
            return node
        current_node = Node(point, point_id)
        depth %= 3

        if node.point[self.__dict_key(depth)] < current_node.point[self.__dict_key(depth)]:
            node.right_node = self.build_kdtree(node.right_node, depth + 1, point, point_id)
        else:
            node.left_node = self.build_kdtree(node.left_node, depth + 1, point, point_id)
        return node

    def search_elements(self, node, search_point, distance_threshold, depth=0, kdtree_search_results=set()):
        """
        :parameter node: node of kdtree
        :parameter search_point: (x,y,z) point
        :parameter distance_threshold: pcd elements near point
        :parameter depth: level of the kdtree depth
        :parameter kdtree_search_results: level of the kdtree depth
        :return: ids which can be considered as near points
        """

        depth %= 3
        current_node = node

        if current_node is not None:
            if(((current_node.point["X"] < search_point[0] + distance_threshold) and (current_node.point["X"] > search_point[0] - distance_threshold)) and
               ((current_node.point["Y"] < search_point[1] + distance_threshold) and (current_node.point["Y"] > search_point[1] - distance_threshold)) and
               ((current_node.point["Z"] < search_point[2] + distance_threshold) and (current_node.point["Z"] > search_point[2] - distance_threshold))):

                point_distance = math.sqrt(math.pow((current_node.point["X"] - search_point[0]), 2) +
                                           math.pow((current_node.point["Y"] - search_point[1]), 2) +
                                           math.pow((current_node.point["Z"] - search_point[2]), 2))
                if point_distance <= distance_threshold:
                    kdtree_search_results.add(current_node.point_id)
                else:
                    pass
            if current_node.point[self.__dict_key(depth)] < search_point[depth] + distance_threshold:
                self.search_elements(current_node.right_node, search_point, distance_threshold,
                                     depth+1, kdtree_search_results)

            if current_node.point[self.__dict_key(depth)] > search_point[depth] - distance_threshold:
                self.search_elements(current_node.left_node, search_point, distance_threshold,
                                     depth+1, kdtree_search_results)
            return kdtree_search_results

    def display_kdtree(self, node, depth=0):
        """
        updates the self.kdtree_dict with the points are corresponding depth
        :parameter node: root node
        :parameter depth: indicates the depth of Kdtree

        """
        current_node = node
        try:
            self.kdtree_display_dict[depth].extend([(current_node.point["X"],
                                                    current_node.point["Y"],
                                                    current_node.point["Z"])])
        except KeyError:
            self.kdtree_display_dict[depth] = [(current_node.point["X"],
                                                current_node.point["Y"],
                                                current_node.point["Z"])]
        if current_node is not None:
            if current_node.left_node is not None:
                left_node = current_node.left_node
                depth += 1
                self.display_kdtree(left_node ,depth)

            if current_node.right_node is not None:
                right_node = current_node.right_node
                depth += 1
                self.display_kdtree(right_node ,depth)

    @staticmethod
    def __dict_key(number):
        """
        returns XYZ based on number
        :param number: 0,1,2
        :return: X,Y,Z
        """
        try:
            key_dict = {0: "X", 1: "Y", 2: "Z"}
            return str(key_dict[number])
        except KeyError:
            raise Exception(f"Incorrect Level({number}) Assignment.")


## Class for processing point cloud data

PointCloudProcessor is for processing point cloud data. It provides a variety of functions for filtering, denoising, segmenting, and visualizing point clouds. PointCloudProcessor can be used for a variety of applications, including object detection, scene segmentation, tracking, and path planning.

In [ ]:

class PointCloudProcessor:
    # Initialize the PointCloudProcessor class.

    # file_path: Path to the input point cloud file (PCD or XYZ format).
    # num_rows: Number of rows (points) to read from the input file.
    # display_output: Boolean flag to control output display.

    def __init__(self, pcd_file, nrows_value, display_output_flag):
        self.nrows = nrows_value
        if pcd_file.endswith(".pcd"):
            pcd = o3d.io.read_point_cloud(r"/content/drive/MyDrive/point_cloud_data_sample_2.pcd")
            self.pcd_data = pd.DataFrame(pcd.points, columns=["X" ,"Y" ,"Z"])
            self.pcd_data = self.pcd_data[0:self.nrows]
        elif pcd_file.endswith(".xyz"):
            self.pcd_data = pd.read_csv("/content/drive/MyDrive/point_cloud_data_sample.xyz", delimiter=" ", nrows=nrows_value)
        else:
            raise Exception(f"Unsupported datatype of pcd file.")
        self.kdtree_main = KdTree_class()
        self.kdtree_root_node = self.kdtree_main.insert_points(self.pcd_data, display_output=display_output_flag)

    def euclidean_clustering(self, distance_threshold, cluster_parameters):
        clusters_identified = dict()
        cluster_id = 0
        processed_flag = [False]*self.nrows
        for point in self.pcd_data.iterrows():
            index = point[0]
            current_point = (point[1]["X"], point[1]["Y"], point[1]["Z"])
            if not processed_flag[index]:
                base_cluster = set()
                self.find_clusters(current_point, base_cluster, index, distance_threshold, cluster_parameters, processed_flag)
                if base_cluster is not None:
                    if (len(base_cluster) > cluster_parameters["min_size"]):
                        clusters_identified[cluster_id] = base_cluster
                        cluster_id += 1
        return clusters_identified

    def get_point(self, index):
        return (self.pcd_data.loc[index]["X"],
                self.pcd_data.loc[index]["Y"],
                self.pcd_data.loc[index]["Z"])


      # base_cluster: Set of points in the current cluster.
      # index: Index of the current point.
      # threshold: Minimum distance for cluster identification.
      # cluster_params: Cluster parameters (e.g., {"min_size": value}).
      # processed_flag: List containing status of points identified as clusters.
    def find_clusters(self, currPoint, baseCluster, ind, threshold, clusterParameters, processed):
        if not processed[ind]:
            processed[ind] = True
            baseCluster.add(ind)
            nearby_points = self.kdtree_main.search_elements(node=self.kdtree_root_node ,search_point=currPoint ,
                                                             distance_threshold=threshold ,depth=0)
            nearby_points_ = copy.copy(nearby_points)
            if len(nearby_points_):
                for ind in nearby_points_:
                    if not processed[ind]:
                        point = self.get_point(ind)
                        self.find_clusters(point, baseCluster, ind,
                                           threshold, clusterParameters, processed)
    def visualize_clusters(self, clusters):
        trace_1 = go.Scatter3d(x=self.pcd_data.X,
                             y=self.pcd_data.Y,
                             z=self.pcd_data.Z,
                             mode='markers')
        cluster_points = pd.DataFrame(columns=["Cluster_id", "X", "Y", "Z"])
        for key in clusters:

            cluster_point_indexes = clusters[key]
            for point_index in cluster_point_indexes:
                point = self.get_point(point_index)
                cluster_points = cluster_points.append({"Cluster_id": key,
                                                         "X": point[0],
                                                         "Y": point[1],
                                                         "Z": point[2]}, ignore_index=True)
        unique_clusters = []
        for cluster_id in cluster_points.Cluster_id.unique():
            cluster_traces = [go.Scatter3d(x=cluster_points[cluster_points["Cluster_id"] == cluster_id].X ,
                                   y=cluster_points[cluster_points["Cluster_id"] == cluster_id].Y ,
                                   z=cluster_points[cluster_points["Cluster_id"] == cluster_id].Z ,
                                   mode='markers')]
            unique_clusters.extend(cluster_traces)
        data = [trace_1]
        data.extend(unique_clusters)
        fig = go.Figure(data)
        fig.show()


APPLICATION = PointCloudProcessor(pcd_file="/content/drive/MyDrive/point_cloud_data_sample_2.pcd" ,nrows_value=1000, display_output_flag=True)
clusters = APPLICATION.euclidean_clustering(distance_threshold=5, cluster_parameters={"min_size":2})
# print(clusters)
APPLICATION.visualize_clusters(clusters)

Kdtree Build Complete
Depth = 0, Points = [(52.30099868774414, 7.300000190734863, 1.9950000047683716)] 
Depth = 1, Points = [(50.571998596191406, 7.21999979019165, 1.937000036239624)] 
Depth = 2, Points = [(24.635000228881836, 6.6539998054504395, 1.0720000267028809)] 
Depth = 3, Points = [(24.415000915527344, 6.677000045776367, 1.065000057220459), (48.78799819946289, 7.2769999504089355, 1.8769999742507935)] 
Depth = 4, Points = [(24.13800048828125, 6.682000160217285, 1.055999994277954), (-36.6349983215332, 7.2179999351501465, 1.472000002861023), (48.75299835205078, 7.427999973297119, 1.8769999742507935)] 
Depth = 5, Points = [(23.905000686645508, 6.698999881744385, 1.0490000247955322), (-38.361000061035156, 5.941999912261963, 1.5219999551773071), (47.98899841308594, 7.466000080108643, 1.8509999513626099), (-71.40699768066406, 35.625999450683594, 2.9079999923706055)] 
Depth = 6, Points = [(23.697999954223633, 6.7210001945495605, 1.0420000553131104), (-70.35099792480469, -2.3610000610351

<ipython-input-6-822e61ecb919>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster_points = cluster_points.append({"Cluster_id": key,
<ipython-input-6-822e61ecb919>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster_points = cluster_points.append({"Cluster_id": key,
<ipython-input-6-822e61ecb919>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster_points = cluster_points.append({"Cluster_id": key,
<ipython-input-6-822e61ecb919>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster_points = cluster_points.append({"Cluster_id": key,
<ipython-input-6-822e61ecb919>:72: FutureWarning: The frame.append method is depreca

## Main driver code for the program

In [ ]:
point_cloud = pd.read_csv("/content/drive/MyDrive/point_cloud_data_sample.xyz", delimiter=" ", nrows=10)

tree = KdTree_class()
root_node = tree.insert_points(point_cloud, display_output=True)
print(tree.search_elements(root_node, (3.12998489, -4.83957614, -6.24208885), 0.5))

Kdtree Build Complete
Depth = 0, Points = [(2.75421051, -4.75981225, -6.44403559)] 
Depth = 1, Points = [(1.07206253, -0.7416798, -5.10408789)] 
Depth = 2, Points = [(-2.84094568, -1.76128496, -6.89372498), (2.96957211, -4.67333497, -6.42598539)] 
Depth = 3, Points = [(2.25035099, -1.21843304, -6.33910852), (-2.4484963, 0.49504791, -4.2454682), (4.93840889, -4.73994435, -7.98122549)] 
Depth = 4, Points = [(3.12998489, -4.83957614, -6.24208885), (4.39719013, -2.57718195, -7.33467585)] 
Depth = 5, Points = [(3.25234417, -4.89076139, -6.10257154)] 
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 11